# Predicting Job Automation Risk with Multilayer Perceptrons (MLPs)

In this tutorial-style notebook, we use a Multilayer Perceptron (MLP) to predict the **job automation risk category** by 2030 using the `AI_Impact_on_Jobs_2030` dataset.

We will:
- Explore the dataset with visualisations.
- Build a preprocessing pipeline for numeric + categorical features.
- Train a baseline MLP classifier for `Risk_Category`.
- Systematically vary **depth** and **width** of the MLP and measure performance.
- Train a tuned MLP and evaluate it.
- Use multiple graphs: heatmaps, confusion matrices, learning curve, PCA plot.


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    cross_val_score,
    learning_curve
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)
from sklearn.decomposition import PCA

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (8, 5)

print('Libraries imported.')

In [ ]:
file_path = '/content/drive/MyDrive/AI_Impact_on_Jobs_2030.csv'  # change if needed
df = pd.read_csv(file_path)

print('Shape:', df.shape)
df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.info()

In [ ]:
risk_counts = df['Risk_Category'].value_counts().sort_index()
print(risk_counts)

plt.figure()
sns.barplot(x=risk_counts.index, y=risk_counts.values)
plt.title('Distribution of Risk Categories')
plt.xlabel('Risk Category')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure()
sns.histplot(df['Automation_Probability_2030'], bins=20, kde=True)
plt.title('Distribution of Automation Probability (2030)')
plt.xlabel('Automation Probability 2030')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure()
sns.boxplot(data=df, x='Risk_Category', y='Average_Salary')
plt.title('Average Salary by Risk Category')
plt.xlabel('Risk Category')
plt.ylabel('Average Salary')
plt.show()

In [ ]:
plt.figure()
sns.scatterplot(
    data=df,
    x='AI_Exposure_Index',
    y='Automation_Probability_2030',
    hue='Risk_Category'
)
plt.title('AI Exposure vs Automation Probability')
plt.xlabel('AI Exposure Index')
plt.ylabel('Automation Probability 2030')
plt.legend(title='Risk Category')
plt.show()

In [ ]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
corr = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=False, cmap='viridis')
plt.title('Correlation Heatmap (Numeric Features)')
plt.show()

In [ ]:
target_col = 'Risk_Category'

feature_cols = [
    'Average_Salary', 'Years_Experience',
    'AI_Exposure_Index', 'Tech_Growth_Factor',
    'Skill_1', 'Skill_2', 'Skill_3', 'Skill_4', 'Skill_5',
    'Skill_6', 'Skill_7', 'Skill_8', 'Skill_9', 'Skill_10',
    'Education_Level'
]

X = df[feature_cols].copy()
y = df[target_col].copy()

categorical_features = ['Education_Level']
numeric_features = [col for col in feature_cols if col not in categorical_features]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print('Train shape:', X_train.shape)
print('Test shape:', X_test.shape)

In [ ]:
# Use sparse_output=False (or sparse=False on older sklearn) to return dense arrays
try:
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

numeric_transformer = StandardScaler()

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

baseline_mlp = MLPClassifier(
    hidden_layer_sizes=(64, 64),
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=300,
    random_state=42
)

baseline_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('mlp', baseline_mlp)
])

baseline_model.fit(X_train, y_train)

y_pred_baseline = baseline_model.predict(X_test)
baseline_acc = accuracy_score(y_test, y_pred_baseline)
print(f'Baseline MLP Test Accuracy: {baseline_acc:.4f}')
print('\nClassification Report (Baseline):')
print(classification_report(y_test, y_pred_baseline))

In [ ]:
labels_sorted = sorted(y.unique())
cm = confusion_matrix(y_test, y_pred_baseline, labels=labels_sorted)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels_sorted)

plt.figure()
disp.plot(values_format='d', cmap='Blues')
plt.title('Confusion Matrix - Baseline MLP')
plt.show()

In [ ]:
depth_options = [1, 2, 3]
width_options = [16, 64, 128]

results = []

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for depth in depth_options:
    for width in width_options:
        layers = tuple([width] * depth)
        mlp = MLPClassifier(
            hidden_layer_sizes=layers,
            activation='relu',
            solver='adam',
            learning_rate_init=0.001,
            max_iter=300,
            random_state=42
        )

        model = Pipeline(steps=[
            ('preprocess', preprocess),
            ('mlp', mlp)
        ])

        scores = cross_val_score(
            model,
            X_train,
            y_train,
            cv=cv,
            scoring='accuracy'
        )

        results.append({
            'depth': depth,
            'width': width,
            'mean_accuracy': scores.mean(),
            'std_accuracy': scores.std()
        })

results_df = pd.DataFrame(results)
results_df.sort_values(by='mean_accuracy', ascending=False).head(10)

In [ ]:
pivot = results_df.pivot(index='depth', columns='width', values='mean_accuracy')

plt.figure(figsize=(8, 5))
sns.heatmap(pivot, annot=True, fmt='.3f', cmap='viridis')
plt.title('Cross-Validated Accuracy for Different MLP Architectures')
plt.xlabel('Width (neurons per hidden layer)')
plt.ylabel('Depth (number of hidden layers)')
plt.show()

In [ ]:
plt.figure()
for width in width_options:
    subset = results_df[results_df['width'] == width].sort_values('depth')
    plt.plot(
        subset['depth'],
        subset['mean_accuracy'],
        marker='o',
        label=f'width={width}'
    )

plt.title('Effect of Depth and Width on Accuracy')
plt.xlabel('Depth (hidden layers)')
plt.ylabel('Mean CV Accuracy')
plt.legend()
plt.show()

In [ ]:
best_row = results_df.sort_values('mean_accuracy', ascending=False).iloc[0]
best_depth = int(best_row['depth'])
best_width = int(best_row['width'])
best_layers = tuple([best_width] * best_depth)

print('Best depth:', best_depth)
print('Best width:', best_width)
print('Best hidden_layer_sizes:', best_layers)

best_mlp = MLPClassifier(
    hidden_layer_sizes=best_layers,
    activation='relu',
    solver='adam',
    learning_rate_init=0.001,
    max_iter=300,
    random_state=42
)

best_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('mlp', best_mlp)
])

best_model.fit(X_train, y_train)

y_pred_best = best_model.predict(X_test)
best_acc = accuracy_score(y_test, y_pred_best)
print(f'Best MLP Test Accuracy: {best_acc:.4f}')
print('\nClassification Report (Best Model):')
print(classification_report(y_test, y_pred_best))

cm_best = confusion_matrix(y_test, y_pred_best, labels=labels_sorted)
disp_best = ConfusionMatrixDisplay(confusion_matrix=cm_best, display_labels=labels_sorted)

plt.figure()
disp_best.plot(values_format='d', cmap='Blues')
plt.title('Confusion Matrix - Best MLP Model')
plt.show()

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(
    best_model,
    X_train,
    y_train,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5),
    shuffle=True
)

train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

plt.figure()
plt.plot(train_sizes, train_mean, marker='o', label='Training accuracy')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2)

plt.plot(train_sizes, val_mean, marker='o', label='Validation accuracy')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.2)

plt.title('Learning Curve - Best MLP Model')
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# PCA visualisation of preprocessed features
X_processed = preprocess.fit_transform(X)

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_processed)

pca_df = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'Risk_Category': y.values
})

plt.figure()
sns.scatterplot(
    data=pca_df,
    x='PC1',
    y='PC2',
    hue='Risk_Category',
    alpha=0.7
)
plt.title('PCA Projection of Jobs Coloured by Risk Category')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Risk Category')
plt.show()